In [138]:
import pandas as pd
df= pd.read_csv("/content/tunisia-real-estate (1).csv")
display(df)

,Unnamed: 0,Delegation,Locality,Nature,Type of Real Estate,Surface,Price,Inserted On
0,0,82,Sidi El Bechir,Sale,2-room apartment,70.0,120000.0,06/10/2023
1,1,12,La Marsa,Sale,3-room apartment,130.0,277000.0,23/10/2023
2,2,28,Ariana Ville,Sale,2-room apartment,111.0,195000.0,09/11/2023
3,3,77,Ettahrir,Sale,5-room apartment and more,150.0,268000.0,13/11/2023
4,4,12,La Marsa,Sale,3-room apartment,134.0,340000.0,08/11/2023
...,...,...,...,...,...,...,...,...
2348,2444,13,La Goulette,Sale,2-room apartment,83.0,330000.0,25/12/2023
2349,2445,41,Ezzahra,Sale,2-room apartment,80.0,225000.0,22/12/2023
2350,2446,12,La Marsa,Sale,3-room apartment,88.0,225000.0,25/12/2023
2351,2447,76,El Mourouj,Sale,3-room apartment,80.0,137000.0,20/12/2023


In [139]:
df=df.drop(columns=["Unnamed: 0","Inserted On","Nature","Locality"], errors="ignore")
display(df)

,Locality,Type of Real Estate,Surface,Price
0,Sidi El Bechir,2-room apartment,70.0,120000.0
1,La Marsa,3-room apartment,130.0,277000.0
2,Ariana Ville,2-room apartment,111.0,195000.0
3,Ettahrir,5-room apartment and more,150.0,268000.0
4,La Marsa,3-room apartment,134.0,340000.0
...,...,...,...,...
2348,La Goulette,2-room apartment,83.0,330000.0
2349,Ezzahra,2-room apartment,80.0,225000.0
2350,La Marsa,3-room apartment,88.0,225000.0
2351,El Mourouj,3-room apartment,80.0,137000.0


⚠️ rows with surface higher then 200 m2 are probably houses not APT [‼️reference](https://edifia-immobilier.com/informations-utiles-achat-appartement/258-lois-promotion-immobiliere-tunisie )

In [140]:
# Delete rows where the surface is greater than 200
df = df[df["Surface"] <= 200]

# Feature engineering

Extract Number of Rooms

In [141]:
# Regular Expression to Extract Number of Rooms
df['rooms'] = df['Type of Real Estate'].str.extract(r'(\d+)-room', expand=False).astype(float)
#drop the Type of Real Estate column
df=df.drop(columns=["Type of Real Estate"], errors="ignore")
display(df)

<ipython-input-141-c5511950b85d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rooms'] = df['Type of Real Estate'].str.extract(r'(\d+)-room', expand=False).astype(float)


,Locality,Surface,Price,rooms
0,Sidi El Bechir,70.0,120000.0,2.0
1,La Marsa,130.0,277000.0,3.0
2,Ariana Ville,111.0,195000.0,2.0
3,Ettahrir,150.0,268000.0,5.0
4,La Marsa,134.0,340000.0,3.0
...,...,...,...,...
2348,La Goulette,83.0,330000.0,2.0
2349,Ezzahra,80.0,225000.0,2.0
2350,La Marsa,88.0,225000.0,3.0
2351,El Mourouj,80.0,137000.0,3.0


In [142]:
#inspect the surface of S+3 APT
display(df[df["rooms"]==3])

,Locality,Surface,Price,rooms
1,La Marsa,130.0,277000.0,3.0
4,La Marsa,134.0,340000.0,3.0
5,Bou Mhel El Bassatine,100.0,125000.0,3.0
6,La Marsa,167.0,467000.0,3.0
8,Ain Zaghouan,138.0,555000.0,3.0
...,...,...,...,...
2341,Bab Bhar,85.0,165000.0,3.0
2342,Ariana Ville,134.0,310000.0,3.0
2344,La Soukra,124.0,347000.0,3.0
2350,La Marsa,88.0,225000.0,3.0


In [143]:
# Filter rows where "surface" is under 100 and Reduce the "rooms" column value to 2
df.loc[df["Surface"] < 100, "rooms"] = 2

In [144]:
# Filter rows where "surface" is under 70 and Reduce the "rooms" column value to 1
df.loc[df["Surface"] < 70, "rooms"] = 1

In [145]:
# As i inspected the surface doesn't match the number of rooms so i can say that they counted the livingroom (salon) as seperate room
df.loc[df['rooms'] == 5, 'rooms'] -= 1
df.loc[(df['rooms'] == 4) & (df['Surface'] < 170) , 'rooms'] -= 1

Extract Number of bathrooms

In [146]:
# Initialize bathrooms column
df['bathrooms'] = 1  # Default value

# Apply the logic
df.loc[(df['rooms'] == 2) & (df['Surface'] > 100), 'bathrooms'] = 2
df.loc[(df['rooms'] == 3) & (df['Surface'] < 150), 'bathrooms'] = 2
df.loc[(df['rooms'] == 3) & (df['Surface'] >= 150), 'bathrooms'] = 3
df.loc[df['rooms'] > 3, 'bathrooms'] = 3

In [147]:
display(df)

,Locality,Surface,Price,rooms,bathrooms
0,Sidi El Bechir,70.0,120000.0,2.0,1
1,La Marsa,130.0,277000.0,3.0,2
2,Ariana Ville,111.0,195000.0,2.0,2
3,Ettahrir,150.0,268000.0,3.0,3
4,La Marsa,134.0,340000.0,3.0,2
...,...,...,...,...,...
2348,La Goulette,83.0,330000.0,2.0,1
2349,Ezzahra,80.0,225000.0,2.0,1
2350,La Marsa,88.0,225000.0,2.0,1
2351,El Mourouj,80.0,137000.0,2.0,1


In [149]:
#inspect the surface of S+4 APT
display(df[df["rooms"]==4])

,Locality,Surface,Price,rooms,bathrooms
21,Ezzahra,175.0,470000.0,4.0,3
34,Ariana Ville,180.0,390000.0,4.0,3
66,La Marsa,179.0,398000.0,4.0,3
70,Ariana Ville,177.0,430000.0,4.0,3
84,Ain Zaghouan,180.0,495000.0,4.0,3
...,...,...,...,...,...
2295,La Marsa,190.0,480000.0,4.0,3
2312,La Marsa,189.0,450000.0,4.0,3
2313,Ariana Ville,182.0,460000.0,4.0,3
2315,La Marsa,200.0,480000.0,4.0,3


# Export Dataset in CSV file

In [150]:
# Reset the index of the DataFrame
df.reset_index(drop=True, inplace=True)

# Export the updated DataFrame to a CSV file
df.to_csv('tunisia-real-estate_final.csv', index=False)
